# Networks

Clustering and visualization of player data is not only based on their numerical features but also on structural (network) features that must be created an prepared beforehand.

__SUBJECT TO CHANGE - CHECK VALUES w/ SUPERVISOR BEFORE FULL DUMP__

## Imports

In [2]:

import os
import configparser
import json
import random
import itertools
import pickle
from pandas import DataFrame
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

from multiprocessing import Process

config = configparser.ConfigParser()
config.read('task_config.ini')

ROOT_DIRPATH = config['GENERAL']['ROOT']
TARGET_DIRPATH = config['GENERAL']['TARGET']
EGO_DIRPATH = ROOT_DIRPATH + config['API']['EGOS']
DICT_FILEPATH = ROOT_DIRPATH + config['API']['DICT']
SURVEY_FILEPATH = ROOT_DIRPATH + config['SURVEY']['COMBINED']
RESULT_FILEPATH = TARGET_DIRPATH + config['RESULT']['PLAYERS']
RESULT_FILE = TARGET_DIRPATH + config['RESULT']['OVERALL']

### Load ID Dictionary

The ID Dictionary maps Summoner IDs to their filepath in the API Data

In [3]:
with open(DICT_FILEPATH, 'rb') as iddict:
    ids = pickle.load(iddict)
with open(SURVEY_FILEPATH, 'rb') as surveyData:
    survey = pickle.load(surveyData)

FileNotFoundError: [Errno 2] No such file or directory: 'Z:/univie/iddict.pickle'

### Save processed player data

A method for storing processed player data in the specified filepath as well as a method for retrieving the right filepath having a summoner id.

In [ ]:
def saveResult(filepath, result):
    if not os.path.exists(os.path.dirname(filepath)):
        try:
            os.makedirs(os.path.dirname(filepath))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    with open(filepath, 'wb') as f:
        pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

def getResultFilepath(summonerId):
    puuid = ids[summonerId].split('/')[1]
    return RESULT_FILEPATH + puuid[0:2] + '/' + puuid + '.pickle'

### Method: Create Player

This method creates Python dictionaries containing details of a specific player and it's connections to other players.

In [ ]:
def create_player(summonerId):
    playerPath = ROOT_DIRPATH + '/' + ids[summonerId] + '/'
    result = {'id': summonerId, 'puuid': playerPath.split("/")[-2], 'alters': dict(), 'stats': dict()}

    with open (playerPath + 'summoner.json', 'r') as summonerinput:
        ego = json.load(summonerinput)
        result['name'] = ego['name']
        result['platform'] = ego['platformId']
        result['level'] = ego['summonerLevel']
        if 'linkId' in ego and int(ego['linkId']) in survey:
            result['surveyData'] = survey[int(ego['linkId'])]
            result['custId'] = result['surveyData']['id']

    kda = []
    kp = []
    teamGold = []
    teamDamage = []
    ckpm = []
    won = 0
    killsWon = 0
    lost = 0
    deathsLost = 0

    for matchfile in os.listdir(playerPath):
        if matchfile in ['summoner.json', 'rankedstats.json', 'matchhistory.json']:
            continue
        with open(playerPath + matchfile, 'r') as matchinput:
            match = json.load(matchinput)
            teams = {str(100): dict(), str(200): dict()}
            teamIds = {str(100): [], str(200): []}
            playerTeam = -1
            totalDamage = 0
            totalGold = 0
            totalKills = 0
            for participant in match['participants']:
                teams[str(participant['teamId'])][participant['participantId']] = participant['stats']
            for partIdendity in match['participantIdentities']:
                if 'player' in partIdendity and 'summonerId' in partIdendity['player']:
                    for key, values in teams.items():
                        if partIdendity['participantId'] in values:
                            if partIdendity['player']['summonerId'] == ego['id']:
                                playerTeam = key
                                egoKills = values[partIdendity['participantId']]['kills']
                                egoAssists = values[partIdendity['participantId']]['assists']
                                egoDeaths = values[partIdendity['participantId']]['deaths']
                                egoDamage = values[partIdendity['participantId']]['totalDamageDealt']
                                egoGold = values[partIdendity['participantId']]['goldEarned']
                                totalKills = totalKills + egoKills
                                totalDamage = totalDamage + egoDamage
                                totalGold = totalGold + egoGold
                                if values[partIdendity['participantId']]['win']:
                                    won = won + 1
                                    killsWon = killsWon + egoKills
                                else:
                                    lost = lost + 1
                                    deathsLost = deathsLost + egoDeaths
                            else:
                                teamIds[key].append((partIdendity['player']['summonerId'], partIdendity['player']['currentPlatformId'], partIdendity['player']['summonerName'], values[partIdendity['participantId']]['kills'], values[partIdendity['participantId']]['totalDamageDealt'], values[partIdendity['participantId']]['goldEarned']))
            if int(playerTeam) > 0:
                for key, values in teamIds.items():
                    for participantTuple in values:
                        result['alters'].setdefault(participantTuple[0], {'platform': participantTuple[1], 'name': participantTuple[2], 'team': [], 'vs': []})['vs' if key != playerTeam else 'team'].append(match['gameId'])
                        if key == playerTeam:
                            totalKills = totalKills + participantTuple[3]
                            totalDamage = totalDamage + participantTuple[4]
                            totalGold = totalGold + participantTuple[5]
                kda.append(((egoKills + egoAssists) / egoDeaths) if egoDeaths else 1)
                if totalKills > 0:
                    kp.append((egoKills + egoAssists) / totalKills)
                if totalGold > 0:
                    teamGold.append(egoGold / totalGold)
                if totalDamage > 0:
                    teamDamage.append(egoDamage / totalDamage)
                if match['gameDuration']:
                    ckpm.append((egoKills + egoDeaths) * 60 / match['gameDuration'])

    if (len(kda)):
        result['stats']['kda'] = sum(kda) / len(kda)
    if (len(kp)):
        result['stats']['kp'] = sum(kp) / len(kp)
    if (len(teamGold)):
        result['stats']['teamGold'] = sum(teamGold) / len(teamGold)
    if (len(teamDamage)):
        result['stats']['teamDamage'] = sum(teamDamage) / len(teamDamage)
    if (len(ckpm)):
        result['stats']['ckpm'] = sum(ckpm) / len(ckpm)
    if won + lost:
        result['stats']['winrate'] = won / (won + lost)
    if won:
        result['stats']['kpw'] = killsWon / won
    if lost:
        result['stats']['dpl'] = deathsLost / lost

    if 'level' not in result:
        return create_player(summonerId)

    return result

### Method: Get Alter

A method for retrieving player details of an alter. It will check the existance of persisted data and return it, otherwise delegates to the create player method for creating and storing it.

In [ ]:
def get_alter(summonerId):
    resultFile = getResultFilepath(summonerId)
    if os.path.exists(resultFile):
        with open(resultFile, 'rb') as f:
            return pickle.load(f)
    result = create_player(summonerId)
    saveResult(resultFile, result)
    return result

### Get Ego by PUUID

Similar to the Get Alter method, a stored copy of a processed object is queried before creating it. Player details of egos also contain all data relevant for 2-level ego networks and their features.

In [ ]:
def get_ego_by_puuid(puuid):
    resultFile = RESULT_FILEPATH + puuid[0:2] + '/' + puuid + '.pickle'
    if os.path.exists(resultFile):
        with open(resultFile, 'rb') as f:
            result = pickle.load(f)
            if 'network' in result:
                return result
            summonerId = result['id']
    else:
        with open (EGO_DIRPATH + puuid + '/' + 'summoner.json', 'r') as summonerinput:
            ego = json.load(summonerinput)
            summonerId = ego['id']
            result = create_player(summonerId)

    if 'custId' not in result:
        saveResult(resultFile, result)
        return result

    result['edgesEgo'] = []
    result['nodesAlters'] = []
    result['edgesAlters'] = []
    result['nodes2nd'] = []
    result['edges2nd'] = []
    
    result['altersTeam'] = []
    result['altersVs'] = []

    result['nodesEgo'] = {"id": summonerId, "value": 30, "label": result['custId']}

    alters = set(result['alters'])
    remainingAlters = alters.copy()
    components = list([a] for a in alters)
    stored2ndLevel = set()
    missingAlters = dict()
    

    for alterId in alters:
        alterInfo = result['alters'][alterId]

        teamCount = len(alterInfo['team'])
        vsCount = len(alterInfo['vs'])
        if teamCount:
            result['altersTeam'].append(alterId)
            result['edgesEgo'].append({"from": summonerId, "to": alterId, "value": teamCount, "title": teamCount, "color": "0000FF"})
        if vsCount:
            result['altersVs'].append(alterId)
            result['edgesEgo'].append({"from": summonerId, "to": alterId, "value": vsCount, "title": vsCount, "color": "FF0000"})

        if alterId not in ids:
            result['nodesAlters'].append({"id": alterId, "value": 10, "label": str(random.randint(1000000, 9999999))})
            for match in alterInfo['team']:
                missingAlters.setdefault(match, {'team': [], 'vs': []})['team'].append(alterId)
            for match in alterInfo['vs']:
                missingAlters.setdefault(match, {'team': [], 'vs': []})['vs'].append(alterId)
            continue
        
        remainingAlters.remove(alterId)
        alter = get_alter(alterId)

        result['nodesAlters'].append({"id": alterId, "value": 10, "label": alter['custId'] if 'custId' in alter else str(random.randint(1000000, 9999999))})
        
        merged = [sublist for sublist in components for a in sublist if a in set(alter['alters'])]
        excl = [sublist for sublist in components if sublist not in merged]
        components = excl + [list(set(element for sublist in merged for element in sublist))]

        commonAlterIds = set(alter['alters']) & remainingAlters

        for alterAlterId in commonAlterIds:
            matchInfo = alter['alters'][alterAlterId]
            #result['alterTies'][str((alterId, alterAlterId))] = {key:matchInfo[key] for key in matchInfo if key == 'team' or key == 'vs'}
            
            teamCount = len(matchInfo['team'])
            vsCount = len(matchInfo['vs'])
            if teamCount:
                result['edgesAlters'].append({"from": alterId, "to": alterAlterId, "value": teamCount, "title": teamCount, "color": "0000FF"})
            if vsCount:
                result['edgesAlters'].append({"from": alterId, "to": alterAlterId, "value": vsCount, "title": vsCount, "color": "FF0000"})

        alters2ndLevel = set(alter['alters']) - alters
        if summonerId in alters2ndLevel:
            alters2ndLevel.remove(summonerId)

        for alter2ndLevelId in alters2ndLevel:
            matchInfo = alter['alters'][alter2ndLevelId]
            if alter2ndLevelId not in stored2ndLevel:
                result['nodes2nd'].append({"id": alter2ndLevelId, "value": 5, "label": str(random.randint(1000000, 9999999))})
                stored2ndLevel.add(alter2ndLevelId)
            
            teamCount = len(matchInfo['team'])
            vsCount = len(matchInfo['vs'])
            if teamCount:
                result['edges2nd'].append({"from": alterId, "to": alter2ndLevelId, "value": teamCount, "title": teamCount, "color": "0000FF"})
            if vsCount:
                result['edges2nd'].append({"from": alterId, "to": alter2ndLevelId, "value": vsCount, "title": vsCount, "color": "FF0000"})

    coopMatches = []
    vsMatches = []
    for matchId, match in missingAlters.items():
        match['team'].sort()
        match['vs'].sort()
        for pair in itertools.combinations(match['team'], 2):
            coopMatches.append(pair)
        for pair in itertools.combinations(match['vs'], 2):
            coopMatches.append(pair)
        for id1 in match['team']:
            for id2 in match['vs']:
                vsPlayers = [id1, id2]
                vsPlayers.sort()
                vsMatches.append(tuple(vsPlayers))
        merged = [sublist for sublist in components for a in sublist if a in set(match['team'] + match['vs'])]
        excl = [sublist for sublist in components if sublist not in merged]
        components = excl + [list(set(element for sublist in merged for element in sublist))]
    
    for playerTuple in set(coopMatches):
        result['edgesAlters'].append({"from": playerTuple[0], "to": playerTuple[1], "value": coopMatches.count(playerTuple), "title": coopMatches.count(playerTuple), "color": "0000FF"})
    for playerTuple in set(vsMatches):
        result['edgesAlters'].append({"from": playerTuple[0], "to": playerTuple[1], "value": vsMatches.count(playerTuple), "title": vsMatches.count(playerTuple), "color": "FF0000"})

    numberOfAlters = len(alters) # = Degree
    numberOfTies = len(result['edgesAlters'])
    meanTieStrength = sum(edge['value'] for edge in result['edgesEgo']) / len(result['edgesEgo'])
    density = numberOfTies / (numberOfAlters * (numberOfAlters - 1))
    componentRatio = (len(components) - 1) / (numberOfAlters - 1)
    fragmentationIndex = 1 - sum(len(component) * (len(component) - 1) for component in components) / (numberOfAlters * (numberOfAlters - 1))

    result["network"] = {"Degree": numberOfAlters, "Mean Tie Strength": meanTieStrength, "Density": density, "Components": len(components), "Component Ratio": componentRatio, "Fragmentation Index": fragmentationIndex}

    saveResult(resultFile, result)
    return result

### Get Ego by Summoner ID

Processes the summoner ID and delegates to the _Get Ego by PUUID_ method.

In [ ]:
def get_ego_by_summonerId(summonerId):
    return get_ego_by_puuid(ids[summonerId].split("/")[1])    

In [70]:
with open(ROOT_DIRPATH + "egomatchnumbers.pickle", 'rb') as f:
    for puuid, alterNumber in pickle.load(f):
        if alterNumber < 30 and alterNumber > 0:
            print(alterNumber)
            print(get_ego_by_puuid(puuid)['puuid'])
            if 'surveyData' in get_ego_by_puuid(puuid):
                print(get_ego_by_puuid(puuid)['name'])
                print(get_ego_by_puuid(puuid)['stats'])
                print(get_ego_by_puuid(puuid)['network'])
            else:
                print('no attached psychological data')

1
-iDHR5WaoqkKRWi3OLNnNPO1BhJS7rBHgFnV44y8UDe8B46nya2P5vLLaim2hSQYAUMjXoNgvUrLog
Eye am Back
{'kda': 3.0, 'kp': 0.42857142857142855, 'teamGold': 0.22425005244388505, 'teamDamage': 0.32288847528486403, 'ckpm': 0.7063440156965337, 'winrate': 1.0, 'kpw': 13.0}
{'Degree': 9, 'Mean Tie Strength': 1.0, 'Density': 0.5277777777777778, 'Components': 1, 'Component Ratio': 0.0, 'Fragmentation Index': 0.0}
12
-Ljl6DxJdXWr1dJMGWJogVtNKq948Olx_orlSIEmT-jom8vBUTDq0KsKyHWiymvi_FEQRc-ELKAaFA
no attached psychological data
18
-MMxWoWgTaFAaXO0lRTw790V6969EVSGuCbMKjqI48JcJDFUXzLRooQmemcZ6G2_IDRxTCn2IAOWpw
soupdaddy24
{'kda': 3.9992724867724854, 'kp': 0.5117067875691264, 'teamGold': 0.18332613996924432, 'teamDamage': 0.16921681504076516, 'ckpm': 0.3517928052924197, 'winrate': 0.5, 'kpw': 6.111111111111111, 'dpl': 6.666666666666667}
{'Degree': 139, 'Mean Tie Strength': 1.1571428571428573, 'Density': 0.03425086018142008, 'Components': 2, 'Component Ratio': 0.007246376811594203, 'Fragmentation Index': 0.21624

In [10]:
results = []
internalIdDict = {}
with open(ROOT_DIRPATH + "egomatchnumbers.pickle", 'rb') as f:
    for puuid, alterNumber in pickle.load(f):
        if alterNumber < 30 and alterNumber > 0:
            ego = get_ego_by_puuid(puuid)
            if 'surveyData' in ego:
                internalIdDict['custId'] = puuid
                print(puuid)
                result = {'level': ego['level']}
                for key, value in ego['stats'].items():
                    result[key] = value
                for key, value in ego['surveyData'].items():
                    result[key] = value
                for key, value in ego['network'].items():
                    result[key] = value
                results.append(result)

-iDHR5WaoqkKRWi3OLNnNPO1BhJS7rBHgFnV44y8UDe8B46nya2P5vLLaim2hSQYAUMjXoNgvUrLog
-MMxWoWgTaFAaXO0lRTw790V6969EVSGuCbMKjqI48JcJDFUXzLRooQmemcZ6G2_IDRxTCn2IAOWpw
-Ryo3AcI8eA2nTbzE4x43qLIqgYr-IrEeUT-Qb7MyAdClFund_QtgAM6ufb0ucB4MTZ40idAb4jnlg
-v0V3JC-a0nX3Ck8BXBWykPQTgTwjDjg8zRYzwuzYsuXw48e1NCeFYTlB62mPo1Rcl_2CKSYrSaQgQ
0SLwyKtmFswkAzQYfiRtS55Ab2J0JP2XJEwq4A-1UCuq_tyq1lcs0y7nT7GOcpjrg2o2Zd9KsWC20Q
28Su6oxm1OCoOWFQ40kjvwPnQ6usShAdSRCRXjt8UAIsKaehueoQIs6-1vFnpvLpTl7myvtHEjtG9Q
30cgmQMaRtsSHw5jsD_Dru1tqj0yFIQR1Ixh2ICCLFJtBCVgx3bZIOc_n00IK-GeNqFUm05yEgTHSw
37PauGd0WWB6cEv69lkPASWLE5k7c-qNp4z3c7TZcHHBt_VuNo028xJ46VOFEp3SEXb5SWrjjog8Hg
3a__GkigbN3vKiRMtbncHKZIZtYE14aTbpJwGaebfAp1bHVsV0tHsTouGHeS3jqioW-Ch8MTmN-rHg
3chHawvW9QWjNv7OYuOrbHIF_mcW6ONc2dBjz601g-QtLF9dK9qdZfka_-DLQkEyKONH2DhmoEmI7w
3GeeZM9HXa1ECd9RobTHH3pHg2i7apBrsHokS-phX-mMIiBt1LJJdIeZRp1AcRBZD9R4uaFZcUFsfA
3Gtw6OBE3I6xyOIcruecg1d3Q2Q-FGW-FG-vZyj59wEWYK9TUumwB88KyLe63u5vrwaQ1uxl3jDgfA
3tF3ByF5ByxP4isar04lQjC4zzMdbuZiEJmr9MHfQyPohJp5qMYD

In [11]:
df = DataFrame(results)
df.head()

,level,kda,kp,teamGold,teamDamage,ckpm,winrate,kpw,id,Age,...,mean_PASSION_hp,mean_PASSION_op,profile,Degree,Mean Tie Strength,Density,Components,Component Ratio,Fragmentation Index,dpl
0,66,3.000000,0.428571,0.224250,0.322888,0.706344,1.0,13.000000,736,18,...,1.8,1.8,4,9,1.000000,0.527778,1,0.000000,0.000000,NaN
1,138,3.999272,0.511707,0.183326,0.169217,0.351793,0.5,6.111111,371,22,...,4.2,1.0,2,139,1.157143,0.034251,2,0.007246,0.216244,6.666667
2,142,2.627768,0.548143,0.225393,0.232548,0.504868,0.5,8.555556,2037,18,...,1.0,5.0,1,150,1.046358,0.032304,3,0.013423,0.509262,8.444444
3,140,2.373599,0.452404,0.180374,0.174563,0.373235,0.5,5.615385,1976,22,...,1.8,1.4,1,201,1.164179,0.023582,1,0.000000,0.000000,7.692308
4,158,1.755495,0.533120,0.199424,0.191514,0.628324,0.5,11.000000,1716,21,...,4.6,1.0,3,18,1.000000,0.235294,2,0.058824,0.529412,14.000000


In [11]:
len(df.index)

235

In [1]:
with open(RESULT_FILE, 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)

NameError: name 'RESULT_FILE' is not defined

In [ ]:
df = df.drop(['Gender'], axis=1)

In [ ]:
sse = []
k_rng = range(1,10)
for k in k_rng:
    km = KMeans(n_clusters=k)
    km.fit(df)
    sse.append(km.inertia_)
plt.xlabel('K')
plt.ylabel('Sum of squared error')
plt.plot(k_rng,sse)

In [ ]:
km = KMeans(n_clusters=3)
y_predicted = km.fit_predict(df)
y_predicted

In [ ]:
df[['level','Degree']].head()


In [ ]:
for re in results:
    print(re)

In [ ]:
results = []
with open(ROOT_DIRPATH + "egomatchnumbers.pickle", 'rb') as f:
    for puuid, alterNumber in pickle.load(f):
        if alterNumber < 10 and alterNumber > 0:
            ego = get_ego_by_puuid(puuid)
            if 'surveyData' in ego:
                results.append('level')
                for key, value in ego['stats'].items():
                    results.append(key)
                for key, value in ego['surveyData'].items():
                    results.append(key)
                for key, value in ego['network'].items():
                    results.append(key)
print(set(results))
                

In [11]:
set(key for item in results for key in item)

{'Age',
 'Component Ratio',
 'Components',
 'Degree',
 'Density',
 'Fragmentation Index',
 'Gender',
 'Mean Tie Strength',
 'PANAS_score_NA',
 'PANAS_score_PA',
 'ckpm',
 'dpl',
 'id',
 'kda',
 'kp',
 'kpw',
 'level',
 'mean_ACH_GOAL_mast_ap',
 'mean_ACH_GOAL_mast_av',
 'mean_ACH_GOAL_perf_ap',
 'mean_ACH_GOAL_perf_av',
 'mean_AMO',
 'mean_EXT',
 'mean_IDE',
 'mean_IMI_enj',
 'mean_IMI_tens',
 'mean_IMO',
 'mean_INJ',
 'mean_INT',
 'mean_PASSION_hp',
 'mean_PASSION_op',
 'mean_PENS_aut',
 'mean_PENS_com',
 'mean_PENS_rel',
 'mean_Vitality',
 'profile',
 'teamDamage',
 'teamGold',
 'winrate'}

In [12]:
results[0]

{'level': 66,
 'kda': 3.0,
 'kp': 0.42857142857142855,
 'teamGold': 0.22425005244388505,
 'teamDamage': 0.32288847528486403,
 'ckpm': 0.7063440156965337,
 'winrate': 1.0,
 'kpw': 13.0,
 'id': 736,
 'Age': 18,
 'Gender': 'A2',
 'mean_AMO': 1.66666666666667,
 'mean_EXT': 1.33333333333333,
 'mean_INJ': 3.33333333333333,
 'mean_IDE': 3.66666666666667,
 'mean_INT': 2.0,
 'mean_IMO': 6.0,
 'mean_IMI_enj': 5.85714285714286,
 'mean_IMI_tens': 3.0,
 'PANAS_score_PA': 28,
 'PANAS_score_NA': 18,
 'mean_PENS_rel': 3.66666666666667,
 'mean_PENS_com': 5.66666666666667,
 'mean_PENS_aut': 5.5,
 'mean_ACH_GOAL_perf_ap': 6.33333333333333,
 'mean_ACH_GOAL_perf_av': 4.5,
 'mean_ACH_GOAL_mast_ap': 5.66666666666667,
 'mean_ACH_GOAL_mast_av': 5.33333333333333,
 'mean_Vitality': 2.71428571428571,
 'mean_PASSION_hp': 1.8,
 'mean_PASSION_op': 1.8,
 'profile': 4,
 'Degree': 9,
 'Mean Tie Strength': 1.0,
 'Density': 0.5277777777777778,
 'Components': 1,
 'Component Ratio': 0.0,
 'Fragmentation Index': 0.0}